In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

In [2]:
df = pd.read_csv('../iot23_test.csv')

In [3]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = pd.get_dummies(df['label']).values

In [4]:
scaler = MinMaxScaler()
scaler.fit(X)
normalized_x = scaler.transform(X)
scaler.fit(Y)
normalized_y = scaler.transform(Y)

In [5]:
class DenseNet(nn.Module):
    def __init__(self, num_classes=6):
        super(DenseNet, self).__init__()

        self.fc1 = nn.Linear(24, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(0.25)

        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.25)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.25)

        self.fc4 = nn.Linear(256, num_classes)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)

        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout2(out)

        out = self.fc3(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.dropout3(out)

        out = self.fc4(out)
        out = self.softmax(out)

        return out



# Convert the data to PyTorch tensors

X = torch.tensor(normalized_x, dtype=torch.float32)
Y = torch.tensor(normalized_y, dtype=torch.float32)

# load the model
model = torch.load('../densenet.pt')

with torch.no_grad():
    model.eval()
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    total = Y.size(0)
    _, normal_y_val = torch.max(Y.data,1)
    correct = (predicted == normal_y_val).sum().item()
    accuracy = correct / total
    print('Accuracy on the validation set: {:.2f}%'.format(accuracy * 100))


Accuracy on the validation set: 49.86%
